In [24]:
## imports 
import pandas as pd
import numpy as np
import yaml
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

# comment these out if you don't have plotnine--not essential here/only used once
import matplotlib.pyplot as plt
# import plotnine
# from plotnine import *

## way to connect to mysql 
## if you need to install
## uncomment this line:
#! pip install mysql-connector-python
import mysql.connector

## function to feed path name to load
## credentials
def load_creds(path: str):
    with open(path, 'r') as stream:
        try:
            creds = yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            print(exc)
    return(creds)

pd.options.display.max_rows = 999
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Preliminary: define connection and read sample of data

In [25]:
## read in creds; change the path name if stored
## elsewhere
creds = load_creds("09_db_cred.yaml")


In [26]:
## connect to the database
cnx = mysql.connector.connect(user=creds['practice_database']['user'], 
                            password=creds['practice_database']['password'],
                            port=creds['practice_database']['port'],
                            database= creds['practice_database']['database'],
                            host = creds['practice_database']['host'])
cnx

In [ ]:
q = """
SELECT 
    *
FROM caseinit
LIMIT 5
"""
pd.read_sql_query(q, cnx)

# Activity 1

1. Create a new column -- `in_chicago` when pulling from the `caseinit` table that takes on the value of "YES" if INCIDENT_CITY = Chicago; "NO" otherwise (which represents incidents in Cook County suburbs outside the city limits);  and pull the table. Use `crosstabs` to confirm that this worked
2. Repeat step 1 but also filter out blank strings (`INCIDENT_CITY` == "")
3. Use `where` to row filter to initiations in Chicago and use group by to find the count of cases diverted and not diverted (`is_in_diversion`); pull the table with those counts
4. Modify the query in step 3 to find the proportion of cases in chicago diverted (hint you made need to use case when in a subquery)
5. Modify the query in step 4 to find the proportion of cases in chicago versus cases not in chicago sent to diversion 


In [12]:
# step 1

query_createvar = """
SELECT
    *,
    CASE 
        WHEN INCIDENT_CITY = 'Chicago' 
        THEN 'YES'
        ELSE 'NO'
END as 'in_chicago'
FROM caseinit
"""
df = pd.read_sql_query(query_createvar, cnx)

In [18]:
#df

In [17]:
# Create a crosstab to confirm the results

df['in_chicago'] = df['in_chicago'].astype('category')
df['INCIDENT_CITY'] = df['INCIDENT_CITY'].astype('category')

pd.crosstab(df.in_chicago, df.INCIDENT_CITY).T.head()
pd.crosstab(df.in_chicago, df.INCIDENT_CITY).T.tail()

In [20]:
## step 2
query_createvar_noblank = """
SELECT 
    INCIDENT_CITY,
    CASE 
        WHEN INCIDENT_CITY = 'Chicago' then 'Yes'
        ELSE 'No'
    END AS in_chicago
FROM caseinit
WHERE INCIDENT_CITY <> ""
"""

varcreate_nb = pd.read_sql_query(query_createvar_noblank, cnx)
#pd.crosstab(varcreate_nb.in_chicago, varcreate_nb.INCIDENT_CITY).T.head(20)
#pd.crosstab(varcreate_nb.in_chicago, varcreate_nb.INCIDENT_CITY).T.tail(15)


In [ ]:
## step 3
query_countdiv = """
SELECT 
    count(*) as count_div, 
    is_in_diversion
FROM caseinit
WHERE INCIDENT_CITY = 'Chicago'
GROUP BY is_in_diversion
"""
count_div = pd.read_sql_query(query_countdiv, cnx)
count_div

In [22]:
## step 4
query_propdiv = """
SELECT 
    avg(is_div_bool) as prop_div
FROM  (
    SELECT 
        *,
        CASE 
            WHEN is_in_diversion = 'True' THEN 1
            ELSE 0
        END is_div_bool
    FROM caseinit) AS d
WHERE INCIDENT_CITY = 'Chicago'
"""
prop_div = pd.read_sql_query(query_propdiv, cnx)
prop_div

,prop_div
0,0.0369


In [29]:
## step 5
query_propdiv_compare = """
SELECT 
    AVG(is_div_bool) as prop_div, 
    in_chicago
FROM  (
    SELECT 
        *,
        CASE 
            WHEN is_in_diversion = 'True' THEN 1
            ELSE 0
        END as is_div_bool,
        CASE 
            WHEN INCIDENT_CITY = 'Chicago' THEN 'Yes'
            ELSE 'No'
        END AS in_chicago
    FROM caseinit) AS d
WHERE INCIDENT_CITY <> ''
GROUP BY in_chicago
"""
#prop_div_compare = pd.read_sql_query(query_propdiv_compare, cnx)
#prop_div_compare

# Activity 2 

1. Use the following crosswalk and the `CASE` variable in the `divert` table to create a new variable `DIVERSION_PROGRAM_TEXT` that spells out the diversion programs
    - DC: Drug Court

    - DDPP: Drug Deferred Prosecution

    - DS: Drug School

    - RJCC: Restorative Justice

    - MHC: Mental Health Court

    - VC: Veteran Court

2. Build on the query from step 1 to filter to Narcotics as the `UPDATED_OFFENSE_CATEGORY` and Black or White defendants (based on race in the diversions table) (hint: you'll need to join with the caseinit table based on case_id and case_participant_id, you can do a inner join to keep only those diverted). Select the case_id, case_participant_id, case, race, and diversion_program_text columns

In [32]:
## step 1; query creating var
dp_type = """
SELECT *,
    CASE 
    WHEN DIVERSION_PROGRAM = 'DC' THEN 'Drug Court'
    WHEN DIVERSION_PROGRAM = 'DDPP' THEN 'Drug Deferred Prosecution'
    WHEN DIVERSION_PROGRAM = 'DS' THEN 'Drug School'
    WHEN DIVERSION_PROGRAM = 'RJCC' THEN 'Restorative Justice'
    WHEN DIVERSION_PROGRAM = 'MHC' THEN 'Mental Health Court'
    WHEN DIVERSION_PROGRAM = 'VC' THEN 'Veteran Court'
ELSE 'Other'
END as DIVERSION_PROGRAM_TEXT
FROM divert
"""
n = pd.read_sql_query(dp_type, cnx)
n
pd.crosstab(n.DIVERSION_PROGRAM,
           n.DIVERSION_PROGRAM_TEXT)

,CASE_ID,CASE_PARTICIPANT_ID,RECEIVED_DATE,OFFENSE_CATEGORY,DIVERSION_PROGRAM,REFERRAL_DATE,DIVERSION_COUNT,PRIMARY_CHARGE_OFFENSE_TITLE,STATUTE,RACE,GENDER,DIVERSION_RESULT,DIVERSION_CLOSED_DATE,DIVERSION_PROGRAM_TEXT
0,506061999169,347644693595,01/01/2011 12:00:00 AM,Retail Theft,DS,06/17/2013 12:00:00 AM,2,RETAIL THEFT,720 ILCS 5/16A-3(a),Black,Male,Graduated,06/17/2013 12:00:00 AM,Drug School
1,506061999169,347644693595,01/01/2011 12:00:00 AM,Retail Theft,VC,08/11/2011 12:00:00 AM,1,RETAIL THEFT,720 ILCS 5/16A-3(a),Black,Male,,,Veteran Court
2,506066271711,347649309720,01/02/2011 12:00:00 AM,Retail Theft,MHC,09/14/2012 12:00:00 AM,1,RETAIL THEFT,720 ILCS 5/16A-3(a),Black,Male,Failed,09/14/2012 12:00:00 AM,Mental Health Court
3,506071709491,347654941393,01/03/2011 12:00:00 AM,Narcotics,MHC,07/27/2018 12:00:00 AM,1,[POSSESSION OF CONTROLLED SUBSTANCE WITH INTEN...,720 ILCS 570/401(d)(i),Black,Female,,,Mental Health Court
4,506071709491,347654987555,01/03/2011 12:00:00 AM,Narcotics,MHC,07/27/2018 12:00:00 AM,1,POSSESSION OF A CONTROLLED SUBSTANCE,720 ILCS 570/402(c),White,Male,,,Mental Health Court
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25490,598008265550,453502003399,09/01/2022 12:00:00 AM,Narcotics,DDPP,08/19/2022 12:00:00 AM,1,POSSESSION OF A CONTROLLED SUBSTANCE,720 ILCS 570/402(c),Black,Male,,,Drug Deferred Prosecution
25491,598008459757,453502188044,09/01/2022 12:00:00 AM,Narcotics,DDPP,08/19/2022 12:00:00 AM,1,POSSESSION OF A CONTROLLED SUBSTANCE,720 ILCS 570/402(c),Black,Male,,,Drug Deferred Prosecution
25492,598009042376,453502557335,09/01/2022 12:00:00 AM,Narcotics,DDPP,08/23/2022 12:00:00 AM,1,POSS CAN/>100-500 GRAM/1ST,720 ILCS 550/4(d),Black,Male,,,Drug Deferred Prosecution
25493,598010013408,453503711366,09/01/2022 12:00:00 AM,Narcotics,DDPP,08/31/2022 12:00:00 AM,1,POSSESSION OF A CONTROLLED SUBSTANCE,720 ILCS 570/402(c),Black,Female,,,Drug Deferred Prosecution


DIVERSION_PROGRAM_TEXT,Drug Court,Drug Deferred Prosecution,Drug School,Mental Health Court,Other,Restorative Justice,Veteran Court
DIVERSION_PROGRAM,,,,,,,
ACT,0,0,0,0,668,0,0
ARI,0,0,0,0,879,0,0
BR9,0,0,0,0,5542,0,0
DC,3530,0,0,0,0,0,0
DDPP,0,5686,0,0,0,0,0
DS,0,0,5705,0,0,0,0
MHC,0,0,0,2188,0,0,0
RJCC,0,0,0,0,0,386,0
SEED,0,0,0,0,166,0,0


In [34]:
## step 2: add the filters 
dp_type_filtered = """
SELECT 
    c.CASE_ID, 
    c.CASE_PARTICIPANT_ID,
    d.RACE, 
    DIVERSION_PROGRAM_TEXT
FROM  (
    SELECT 
        *,
        CASE 
            WHEN DIVERSION_PROGRAM = 'DC' THEN 'Drug Court'
            WHEN DIVERSION_PROGRAM = 'DDPP' THEN 'Drug Deferred Prosecution'
            WHEN DIVERSION_PROGRAM = 'DS' THEN 'Drug School'
            WHEN DIVERSION_PROGRAM = 'RJCC' THEN 'Restorative Justice'
            WHEN DIVERSION_PROGRAM = 'MHC' THEN 'Mental Health Court'
            WHEN DIVERSION_PROGRAM = 'VC' THEN 'Veteran Court'
            ELSE 'Other'
        END as DIVERSION_PROGRAM_TEXT
    FROM divert) AS d
INNER JOIN (
    SELECT 
        UPDATED_OFFENSE_CATEGORY,
        CASE_ID, 
        CASE_PARTICIPANT_ID
    FROM caseinit
    ) AS c
ON 
    c.CASE_ID = d.CASE_ID AND
    c.CASE_PARTICIPANT_ID = d.CASE_PARTICIPANT_ID
WHERE d.RACE IN ("Black", "White")
AND UPDATED_OFFENSE_CATEGORY = 'Narcotics'
"""

d_filtered = pd.read_sql_query(dp_type_filtered, cnx)
d_filtered.head()


,CASE_ID,CASE_PARTICIPANT_ID,RACE,DIVERSION_PROGRAM_TEXT
0,506071709491,347654987555,White,Mental Health Court
1,506071709491,347654987555,White,Mental Health Court
2,506106860859,347693763008,Black,Drug School
3,506131913491,347722752275,White,Drug Court
4,506131913491,347722752275,White,Drug Court
